In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_Google/"
#path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_all_Google/"
cost_path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/cost/cost_Google9.csv"

bandwidth ={}
latency  = {}
bw = []
lt = []

listUse=['frankfurt_google', 'london_google', 'mumbai_google', 'northvirginia_google', 'oregon_google', 'saopaulo_google', 'singapore_google', 'sydney_google', 'tokyo_google']

In [102]:
for i in listUse:
    for j in listUse: 
        if i is j:
            bw.append(0)
            lt.append(0)
            continue
        
        # bandwidth
        if "_aws" in j:
            location = path+"%s/combined_bandwidth_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_bandwidth_google_%s_%s.txt"%(i,j,i)
        
        df_bandwidth = pd.read_csv(location, sep='\s+', skiprows=3, nrows=100, header=None)
        headers = ["0", "1", "time","time_unit","transfer","transfer_unit", "bitrate", "bitrate_unit", "retr", "cwnd", "cwnd_unit"]
        df_bandwidth.columns = headers     
        df_bandwidth.drop(['0', '1', 'time', 'time_unit','transfer', 'transfer_unit','cwnd', 'cwnd_unit', 'retr'], axis=1, inplace= True)
        val = df_bandwidth["bitrate"].mean()
        
        bandwidth[j+"_"+i]=val
        bw.append(val)
  
        # latency
        if "_aws" in j:
            location = path+"%s/combined_latency_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_latency_google_%s_%s.txt"%(i,j,i)
        
        df_lat = pd.read_csv(location, sep='\s+', skiprows=1, nrows=100, header=None)

        headers_lat = ["0", "1", "2", "3", "4", "5", "latency", "unit"]
        df_lat.columns = headers_lat
        df_lat.drop(['0', '1', '2', '3','4', '5'], axis=1, inplace= True)        
        df_lat["latency"] = df_lat["latency"].map(lambda x:x.replace("time=",""))
        df_lat["latency"] = df_lat["latency"].astype('float')
        #print(df_lat["latency"].max())
        val_lat = df_lat["latency"].mean()
        
        latency[j+"_"+i]=val_lat
        lt.append(val_lat)

In [103]:
df_cost = pd.read_csv(cost_path, sep=';')
df_cost.head()

,from,frankfurt_google,london_google,mumbai_google,northvirginia_google,oregon_google,saopaulo_google,singapore_google,sydney_google,tokyo_google
0,frankfurt_google,0.000,0.082,0.082,0.082,0.082,0.082,0.082,0.152,0.082
1,london_google,0.082,0.000,0.082,0.082,0.082,0.082,0.082,0.152,0.082
2,mumbai_google,0.082,0.082,0.000,0.082,0.082,0.082,0.082,0.152,0.082
3,northvirginia_google,0.082,0.082,0.082,0.000,0.082,0.082,0.082,0.152,0.082
4,oregon_google,0.082,0.082,0.082,0.082,0.000,0.082,0.082,0.152,0.082


In [104]:
df_cost.drop(['from'], axis = 1, inplace = True)
df_cost.head()

,frankfurt_google,london_google,mumbai_google,northvirginia_google,oregon_google,saopaulo_google,singapore_google,sydney_google,tokyo_google
0,0.000,0.082,0.082,0.082,0.082,0.082,0.082,0.152,0.082
1,0.082,0.000,0.082,0.082,0.082,0.082,0.082,0.152,0.082
2,0.082,0.082,0.000,0.082,0.082,0.082,0.082,0.152,0.082
3,0.082,0.082,0.082,0.000,0.082,0.082,0.082,0.152,0.082
4,0.082,0.082,0.082,0.082,0.000,0.082,0.082,0.152,0.082


In [105]:
# bandwidth
arrayBw = np.array(bw)
# arrayBw= arrayBw.reshape(-1,len(dirList))
arrayBw= arrayBw.reshape(-1,len(listUse))

max_bd = np.max(arrayBw)
min_bd = np.amin(arrayBw[arrayBw>0.00])

# normalization
arrayBw = (arrayBw-min_bd)/(max_bd-min_bd)
arrayBw = arrayBw.round(4)
#arrayBw

In [106]:
# latency
arrayLt = np.array(lt)
arrayLt = arrayLt.reshape(-1,len(listUse))

max_lt = np.max(arrayLt)
min_lt = np.amin(arrayLt[arrayLt>0.00])

# normalization
arrayLt = 1-(arrayLt-min_lt)/(max_lt-min_lt)
arrayLt = arrayLt.round(4)
#arrayLt

In [107]:
# cost
arrayCost = df_cost.values

# Get maximum and minimum
max_cost = np.max(arrayCost) 
min_cost = np.min(arrayCost[arrayCost>0])

# Normalize the values
arrayCost = 1 - (arrayCost - min_cost)/(max_cost - min_cost)
arrayCost = arrayCost.round(4)
#arrayCost

In [108]:
print("Max bandwidth is {}, min bandwidth is {} \nMax latency is {}, min latency is {}\nMax cost is {}, min cost is {}".format(max_bd, min_bd, max_lt, min_lt, max_cost, min_cost))

Max bandwidth is 557.3130000000001, min bandwidth is 8.765 
Max latency is 381.28, min latency is 12.765000000000004
Max cost is 0.152, min cost is 0.08199999999999999


In [109]:
# convert to 0 for i == j
arrayBw = np.where(arrayBw==-0.016, 0, arrayBw)
arrayLt = np.where(arrayLt==1.0346, 0, arrayLt)
arrayCost = np.where(arrayCost==2.1714, 0, arrayCost)

In [110]:
# combine latency, bandwidth, and cost with equal weights to all
# combined = (arrayBw+arrayLt+arrayCost)/3
# combined = 0.25 * arrayBw + 0.25 * arrayLt + 0.5 * arrayCost
# combined = 0.2 * arrayBw + 0.2 * arrayLt + 0.6 * arrayCost
# combined = 0.1 * arrayBw + 0.1 * arrayLt + 0.8 * arrayCost
# combined = 0.0 * arrayBw + 0.0 * arrayLt + 1.0 * arrayCost

# combine both latency and bandwidth
# combined = (arrayBw+arrayLt)/2

# only latency
# combined = arrayLt

# only bandwidth
combined = arrayBw

print(combined.shape)

(9, 9)


In [111]:
df = pd.DataFrame(combined)
df

,0,1,2,3,4,5,6,7,8
0,0.0000,1.0000,0.0463,0.0984,0.1226,0.0088,0.0875,0.0811,0.1012
1,0.1663,0.0000,0.0691,0.1147,0.1021,0.0135,0.0920,0.0958,0.0536
2,0.0759,0.0789,0.0000,0.0952,0.1281,0.0064,0.1782,0.1819,0.0705
3,0.0419,0.0688,0.0954,0.0000,0.2755,0.0125,0.1280,0.1341,0.1865
4,0.1023,0.0578,0.0729,0.0991,0.0000,0.0096,0.1279,0.1513,0.0433
5,0.0834,0.1274,0.0632,0.2133,0.1603,0.0000,0.0651,0.0828,0.0692
6,0.0437,0.0830,0.1646,0.1281,0.1781,0.0036,0.0000,0.3117,0.0916
7,0.0109,0.0274,0.0637,0.0650,0.1209,0.0000,0.1821,0.0000,0.0330
8,0.0848,0.1233,0.0952,0.1808,0.2365,0.0092,0.2996,0.2478,0.0000


In [112]:
#df = pd.DataFrame(combined, index=dirList, columns=dirList)
df = pd.DataFrame(combined)
df.columns = listUse
new_column = listUse
df.insert(0, column="From", value=new_column)

df = df.round(4)
df

,From,frankfurt_google,london_google,mumbai_google,northvirginia_google,oregon_google,saopaulo_google,singapore_google,sydney_google,tokyo_google
0,frankfurt_google,0.0000,1.0000,0.0463,0.0984,0.1226,0.0088,0.0875,0.0811,0.1012
1,london_google,0.1663,0.0000,0.0691,0.1147,0.1021,0.0135,0.0920,0.0958,0.0536
2,mumbai_google,0.0759,0.0789,0.0000,0.0952,0.1281,0.0064,0.1782,0.1819,0.0705
3,northvirginia_google,0.0419,0.0688,0.0954,0.0000,0.2755,0.0125,0.1280,0.1341,0.1865
4,oregon_google,0.1023,0.0578,0.0729,0.0991,0.0000,0.0096,0.1279,0.1513,0.0433
5,saopaulo_google,0.0834,0.1274,0.0632,0.2133,0.1603,0.0000,0.0651,0.0828,0.0692
6,singapore_google,0.0437,0.0830,0.1646,0.1281,0.1781,0.0036,0.0000,0.3117,0.0916
7,sydney_google,0.0109,0.0274,0.0637,0.0650,0.1209,0.0000,0.1821,0.0000,0.0330
8,tokyo_google,0.0848,0.1233,0.0952,0.1808,0.2365,0.0092,0.2996,0.2478,0.0000


In [113]:
#df.max(axis=1)

In [114]:
df.to_csv("lt_bd_cost.csv", sep="\t")

In [60]:
new_df = pd.melt(df, id_vars=['From'], value_vars=df.columns.values.tolist()[1:],
    var_name='To', value_name='Value')

In [164]:
new_df.head()

,From,To,Value
0,frankfurt_aws,frankfurt_aws,1.0000
1,frankfurt_google,frankfurt_aws,0.8140
2,london_aws,frankfurt_aws,0.6550
3,london_google,frankfurt_aws,0.5166
4,mumbai_aws,frankfurt_aws,0.3951


In [165]:
new_df.to_csv("values.csv", sep=",")

In [166]:
dirList

['frankfurt_aws',
 'frankfurt_google',
 'london_aws',
 'london_google',
 'mumbai_aws',
 'mumbai_google',
 'northvirginia_aws',
 'northvirginia_google',
 'oregon_aws',
 'oregon_google',
 'saopaulo_aws',
 'saopaulo_google',
 'singapore_aws',
 'singapore_google',
 'sydney_aws',
 'sydney_google',
 'tokyo_aws',
 'tokyo_google']